In [ ]:
import os
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd
import pdfplumber

# 1 Data download

In [3]:
base_url = "https://oig.hhs.gov/fraud/medicaid-fraud-control-units-mfcu/"
download_dir = "mfcu_charts"
os.makedirs(download_dir, exist_ok=True)

In [12]:
response = requests.get(base_url)
soup = BeautifulSoup(response.content, "lxml")

In [13]:
links = []
for link in soup.find_all("a", href=True):
    href = link["href"]
    text = link.get_text(strip=True)

    if "fy" in text.lower() and "chart" in text.lower():
        file_url = urljoin(base_url, href)
        file_name = file_url.split("/")[-1]
        file_path = os.path.join(download_dir, file_name)
        print(file_url)
        links.append((file_url, file_path))

        try:
            file_data = requests.get(file_url)
            with open(file_path, "wb") as f:
                f.write(file_data.content)
            print(f"✅ Already Download: {file_name}")
        except Exception as e:
            print(f"❌ Failed: {file_url},Error: {e}")

https://oig.hhs.gov/documents/evaluation/10223/FY2024%20Statistical%20Chart_Table_protected.xlsx
✅ Already Download: FY2024%20Statistical%20Chart_Table_protected.xlsx
https://oig.hhs.gov/documents/evaluation/10392/fy2023-statistical-chart.pdf
✅ Already Download: fy2023-statistical-chart.pdf
https://oig.hhs.gov/documents/evaluation/10393/fy2022-statistical-chart.pdf
✅ Already Download: fy2022-statistical-chart.pdf
https://oig.hhs.gov/documents/evaluation/10394/fy2021-statistical-chart.pdf
✅ Already Download: fy2021-statistical-chart.pdf
https://oig.hhs.gov/documents/evaluation/10395/fy2020-statistical-chart.pdf
✅ Already Download: fy2020-statistical-chart.pdf
https://oig.hhs.gov/documents/evaluation/10396/fy2019-statistical-chart.pdf
✅ Already Download: fy2019-statistical-chart.pdf
https://oig.hhs.gov/documents/evaluation/10397/fy2018-statistical-chart.pdf
✅ Already Download: fy2018-statistical-chart.pdf
https://oig.hhs.gov/documents/evaluation/10398/fy2017-statistical-chart.pdf
✅ Alrea

# Format transformation

### html -> csv

In [ ]:
for filename in os.listdir(download_dir):
    if filename.endswith(".htm") or filename.endswith(".html"):
        file_path = os.path.join(download_dir, filename)
        with open(file_path, "r", encoding="utf-8") as f:
            soup = BeautifulSoup(f, "lxml")
        
        # find each table in the HTML file
        tables = soup.find_all("table")
        for i, table in enumerate(tables):
            rows = []
            for row in table.find_all('tr'):
                cols = row.find_all(['td', 'th'])
                cols_text = [col.get_text(strip=True) for col in cols]
                rows.append(cols_text)
            
            df = pd.DataFrame(rows)
            
            df.dropna(how='all', inplace=True)
            df.dropna(axis=1, how='all', inplace=True)
            
            df.to_csv(f'{filename[:-4]}_table{i+1}.csv', index=False, header=False)
            print(f"✅ Table {i+1} from {filename} saved as CSV.")
            os.remove(file_path)  

### xlsx -> csv

In [6]:
for filename in os.listdir(download_dir):
    if filename.endswith(".xlsx") or filename.endswith(".xls"):
        file_path = os.path.join(download_dir, filename)
        df = pd.read_excel(file_path, engine='openpyxl')
        
        # Save as CSV
        csv_file_path = file_path.replace('.xlsx', '.csv')
        csv_file_path = csv_file_path.replace('.xls', '.csv')
        csv_file_path = csv_file_path.replace('%', '_')
        csv_file_path = csv_file_path.lower()
        
        df.to_csv(csv_file_path, index=False)
        print(f"✅ Converted {filename} to CSV: {csv_file_path}")
        
        # Remove the original xlsx file
        os.remove(file_path)

✅ Converted FY2024%20Statistical%20Chart_Table_protected.xlsx to CSV: mfcu_charts\FY2024_20Statistical_20Chart_Table_protected.csv


## pdf -> csv

In [21]:

custom_header_17 = pd.read_csv("mfcu_header_17.csv").columns.tolist()
custom_header_18 = pd.read_csv("mfcu_header_18.csv").columns.tolist()

for filename in os.listdir(download_dir):
    if filename.endswith(".pdf"):
        pdf_path = os.path.join(download_dir, filename)
        df_all = pd.DataFrame()
        with pdfplumber.open(pdf_path) as pdf:
            for i, page in enumerate(pdf.pages):
                tables = page.extract_tables()
                for j, table in enumerate(tables):
                    df = pd.DataFrame(table)
                    if df.empty:
                        df_all = df
                    else:
                        df_all = pd.concat([df_all, df], ignore_index=True)

        #step 1: find ALABAMA
        df_upper = df_all.fillna('').astype(str).applymap(lambda x: x.upper())

        # find row and column where "ALABAMA" appears
        matches = (df_upper == "ALABAMA")

        if matches.any().any():
            first_row, first_col = matches.stack()[matches.stack()].index[0]
        else:
            raise ValueError("cannot find Alabama")
        
        df_clean = df_all.iloc[first_row:, first_col:].reset_index(drop=True)

        # step 2: replace header

        ## check if custom header matches the number of columns
        if len(df_clean) == 17:
            df_clean.columns = custom_header_17
        elif len(df_clean) == 18:
            df_clean.columns = custom_header_18
        else:
            raise ValueError(f"❌ number of columns for Header and data {df_clean.shape[1]} are not equal. Please check your header file.")

        # step 3: save to csv
        csv_file_path = pdf_path.replace('.pdf', '.csv')
        df_clean.to_csv(csv_file_path, index=False)
                        
        print(f"✅ Converted {filename} to CSV: {csv_file_path}")
        # Remove the original PDF file
        os.remove(pdf_path)

C:\Users\木木小呆猪\AppData\Local\Temp\ipykernel_23184\1362482563.py:19: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_upper = df_all.fillna('').astype(str).applymap(lambda x: x.upper())


✅ Converted fy2015-statistical-chart.pdf to CSV: mfcu_charts\fy2015-statistical-chart.csv
✅ Converted fy2016-statistical-chart.pdf to CSV: mfcu_charts\fy2016-statistical-chart.csv


C:\Users\木木小呆猪\AppData\Local\Temp\ipykernel_23184\1362482563.py:19: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_upper = df_all.fillna('').astype(str).applymap(lambda x: x.upper())
C:\Users\木木小呆猪\AppData\Local\Temp\ipykernel_23184\1362482563.py:19: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_upper = df_all.fillna('').astype(str).applymap(lambda x: x.upper())


✅ Converted fy2017-statistical-chart.pdf to CSV: mfcu_charts\fy2017-statistical-chart.csv


C:\Users\木木小呆猪\AppData\Local\Temp\ipykernel_23184\1362482563.py:19: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_upper = df_all.fillna('').astype(str).applymap(lambda x: x.upper())


...

C:\Users\木木小呆猪\AppData\Local\Temp\ipykernel_23184\1362482563.py:19: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_upper = df_all.fillna('').astype(str).applymap(lambda x: x.upper())


✅ Converted fy2021-statistical-chart.pdf to CSV: mfcu_charts\fy2021-statistical-chart.csv


C:\Users\木木小呆猪\AppData\Local\Temp\ipykernel_23184\1362482563.py:19: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_upper = df_all.fillna('').astype(str).applymap(lambda x: x.upper())


✅ Converted fy2022-statistical-chart.pdf to CSV: mfcu_charts\fy2022-statistical-chart.csv
✅ Converted fy2023-statistical-chart.pdf to CSV: mfcu_charts\fy2023-statistical-chart.csv


C:\Users\木木小呆猪\AppData\Local\Temp\ipykernel_23184\1362482563.py:19: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_upper = df_all.fillna('').astype(str).applymap(lambda x: x.upper())
